import libarary

In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime,timedelta

from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [2]:
API_KEY = '6ec1e594af2c807c39df656e481e1441'
base_url = 'https://api.openweathermap.org/data/2.5/'

1)fetch current weather data

In [3]:
def get_wether_data(city):
  url = f"{base_url}weather?q={city}&appid={API_KEY}&units=metric"  # our api request

  response = requests.get(url)
  data=response.json()

  return {
      'city' : data['name'],
      'current_temp' :round(data['main']['temp']),
      'feels_like' : round(data['main']['temp']),
      'temp_min'   : round(data['main']['temp_min']),
      'temp_max'   : round(data['main']['temp_max']),
      'humidity'   : round(data['main']['humidity']),
      'descripton' : data['weather'][0]['description'],
      'country'   : data['sys']['country']
  }

In [4]:
def read_hist_data(file):
  df = pd.read_csv(file)
  df= df.dropna() #any
  df = df.drop_duplicates
  return df



prepare training data

In [5]:
def prepare(data):
    le = LabelEncoder()
    oe = OneHotEncoder(sparse_output=False)  # Ensure output is 2d numpy array

    # Apply OneHotEncoder to WindGustDir
    wind_gust_encoded = oe.fit_transform(data[['WindGustDir']])  # Keep 2D input
    wind_gust_df = pd.DataFrame(wind_gust_encoded, columns=oe.get_feature_names_out(['WindGustDir']))

    # Drop the original WindGustDir column and merge the new columns
    data = data.drop('WindGustDir', axis=1)
    data = pd.concat([data, wind_gust_df], axis=1)

    data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])
    return data

    x = data.drop(['RainTomorrow'])
    y = data['RainTomorrow']

    return x,y,le,oe

now we do experimennts with grid search cv to find the best model and parameter


In [6]:
data = pd.read_csv('weather.csv')
print(data.columns)

le = LabelEncoder()
oe = OneHotEncoder(sparse_output=False)  # Ensure output is 2d numpy array

# Apply OneHotEncoder to WindGustDir
wind_gust_encoded = oe.fit_transform(data[['WindGustDir']])  # Keep 2D input
wind_gust_df = pd.DataFrame(wind_gust_encoded, columns=oe.get_feature_names_out(['WindGustDir']))

# Drop the original WindGustDir column and merge the new columns
data = data.drop('WindGustDir', axis=1)
data = pd.concat([data, wind_gust_df], axis=1)

data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])


# Drop the target column 'RainTomorrow' from feature set
x = data.drop('RainTomorrow', axis=1)  # Corrected axis=1 to drop column

# Target variable
y = data['RainTomorrow']
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state = 42)

FileNotFoundError: [Errno 2] No such file or directory: 'weather.csv'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the model
model = RandomForestClassifier()

# Define the parameter grid
param_distributions = {
    'n_estimators': [80, 100, 120],
    'max_depth': [ 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Instantiate RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the model
random_search.fit(x_train, y_train)

# Output results
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Define the parameter grid for XGBoost
param_distributions = {
    'n_estimators': [80, 100, 120],
    'max_depth': [10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 0.1, 0.2]
}

# Instantiate RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the model
random_search.fit(x_train, y_train)

# Output results
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


we have decided to use random forest with parameters



Best Parameters: {'n_estimators': 120, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': False}

In [ ]:
def train_rain(x,y):
  x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state = 42)


  # Define the model with the specified hyperparameters
  model = RandomForestClassifier(
        n_estimators=120,
        min_samples_split=10,
        min_samples_leaf=1,
        max_depth=30,
        bootstrap=False
    )

  # Fit the model on the training data
  model.fit(x_train, y_train)

  y_pred = model.predict(x_test)

  print('mean square error for model')
  print(mean_squared_error(y_test,y_pred))

  return  model

prepare regression data

In [ ]:
def make_regression_data(data,feature):
  x,y = [],[]
  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  x= np.array(x).reshape(-1,1)
  y = np.array(y).reshape(-1,1)

  return x,y